## Ordinal Classification
### Model Serving

In [18]:
import pandas as pd

# Features verification
from feature_engine.preprocessing import MatchVariables
# Categories verification
from feature_engine.preprocessing import MatchCategories
# Script Serving
from jpmml_evaluator import make_evaluator
from jpmml_evaluator.py4j import Py4JBackend
# REST Serving
from openscoring import Openscoring

In [19]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
# Data Import
df = pd.read_csv('../../../data/winequality-white-cleaned.csv',sep=';')
# Get sample dataset for model serving
df_serve = df.sample(30)
x_serve = df_serve.drop('target', axis=1)
y_serve = df_serve[['target']]
x_serve.reset_index(drop=True, inplace=True)
y_serve.reset_index(drop=True, inplace=True)

## Features Verification

In [21]:
# Ensures that columns in serving set match those in train set
x_train = df_serve.drop('target', axis=1)
# Setup the transformer
match_cols = MatchVariables(missing_values="ignore")
match_cols.fit(x_train)
match_cols.feature_names_in_

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']

In [22]:
# Run the transformer
x_match = match_cols.transform(x_serve)
x_match.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.028196,0.274361,0.461278,2.113835,0.030420,29.994745,108.978981,0.990207,3.375744,0.447285,12.828196
1,6.316556,0.258596,0.435868,3.014437,0.029826,7.686092,106.313908,0.991259,3.461815,0.407272,11.844636
2,8.524475,0.373857,0.404240,1.268480,0.035027,5.000000,48.782443,0.993019,2.995216,0.450976,10.126081
3,9.647888,0.336479,0.652254,3.439440,0.049690,17.929580,120.549300,0.996386,3.079718,0.634225,10.176056
4,6.589248,0.286775,0.310000,3.867745,0.027323,38.677453,95.569937,0.990340,3.237850,0.602150,12.600000


## Categories Verification

In [23]:
# Ensures categorical variables are of type ‘category’
mc = MatchCategories(missing_values="ignore")
# mc.fit(x_train)

In [24]:
# Run the transformer
# x_match_c = mc.transform(x_match)
# x_match_c.head(5)

## Model Script Serving

In [25]:
# Load PMML model
evaluator = make_evaluator(backend=Py4JBackend(), path="../../../data/OrdinalLogisticRegression.pmml")
# Perform automated QA
evaluator.verify()

In [26]:
# Model Serving
dfresult = evaluator.evaluateAll(x_match)
dfresult.head(5)

,target,probability(3),probability(4),probability(5),probability(6),probability(7),probability(8),probability(9)
0,9,0.000453,0.000076,0.000013,0.000092,0.004325,0.010304,9.847376e-01
1,9,0.055264,0.055466,0.012795,0.014762,0.193000,0.317745,3.509679e-01
2,4,0.212337,0.694939,0.081091,0.004804,0.005776,0.001053,2.388514e-08
3,4,0.470142,0.503053,0.019551,0.001658,0.003330,0.001925,3.397066e-04
4,8,0.002723,0.001848,0.000525,0.007999,0.376441,0.603058,7.406137e-03


## Model REST Serving

In [27]:
# Creating an Openscoring object
# Openscoring server have to be running on base-url
os = Openscoring(base_url = "http://localhost:8080", token="secret")

In [28]:
# Deploying a PMML model:
model_name = 'OrdinalLogisticRegression'
os.deployFile(model_name, "../../../data/OrdinalLogisticRegression.pmml")

In [29]:
# Model serving with data records from DataFrame
dfResponse = os.evaluateCsv(model_name, x_match)
print(dfResponse.head(5))

   target  probability(3)  probability(4)  probability(5)  probability(6)  \
0       9        0.000453        0.000076        0.000013        0.000092   
1       9        0.055264        0.055466        0.012795        0.014762   
2       4        0.212337        0.694939        0.081091        0.004804   
3       4        0.470142        0.503053        0.019551        0.001658   
4       8        0.002723        0.001848        0.000525        0.007999   

   probability(7)  probability(8)  probability(9)  
0        0.004325        0.010304    9.847376e-01  
1        0.193000        0.317745    3.509679e-01  
2        0.005776        0.001053    2.388514e-08  
3        0.003330        0.001925    3.397066e-04  
4        0.376441        0.603058    7.406137e-03  
